# Zurich open data exploration

## Reading csv, json and shape format datasets. Combining them into one central file, and visualizing some elements.

One of the goals of our project is to study urban environment of Zurich city and afterwards associate it with insurance data. For this purpose, we will be utilizing public datasets published on <a href="url">https://data.stadt-zuerich.ch/</a> website. Datasets we will be using contain, but are not limited to, public parks, schools, street lights, public WCs and etc. in Zurich. 

This notebook is used to load all relevant datasets found on https://data.stadt-zuerich.ch/ as well as the population per zip code data from https://opendata.swiss/en/dataset/bevoelkerung-pro-plz into a central file that will be used for the rest of the analysis.

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from os import listdir
import matplotlib.pyplot as plt
from shapely.geometry import Point
import pyproj

In [ ]:
DATA_FOLDER = "./data/"
CSV_JSON_FOLDER = DATA_FOLDER + './csv_json_files/'
SHAPE_FOLDER = DATA_FOLDER + "shapeFiles/"

## Reading and loading the csv files

In [ ]:
# create function that reads cleans and appends csv to the general dataframe
def csv_read_clean_append(csv_file, duplicate_check, group_on, new_column, append_target=None):
    
    # load csv into dataframe, drop any duplacte rows, count values for group_on series 
    temp_df = pd.read_csv(CSV_JSON_FOLDER + csv_file).drop_duplicates(duplicate_check)[group_on].value_counts().to_frame(new_column)
    
    if append_target is None:
        return temp_df
    else:
        return append_target.join(temp_df)

In [ ]:
# start with addresses and number of hospitality companies
combined_df = csv_read_clean_append('adressen.csv', 'adresse', 'plz', 'addresses')
combined_df = csv_read_clean_append('gastwirtschaftsbetriebe_per_20171231.csv',\
                                    'Betriebsname', 'plz', 'hospitality_companies', combined_df)

We now insert the population per zip code. We have to do some manual touching up, as the population is a sum of number of women and men.

In [ ]:
# read population per zip code file
population_df = pd.read_csv(CSV_JSON_FOLDER +'bevoelkerung_proplz.csv',delimiter=';')

# sum the men and women count per zip code to get population per zip code
population_df = population_df[population_df['typ']\
                                .isin(['w', 'm'])]\
                                .groupby('plz')['anzahl']\
                                .agg('sum')\
                                .to_frame('population')

# merge the population data to the rest
combined_df = pd.merge(combined_df, population_df, 'inner', left_index=True, right_index=True)
combined_df.head()

## Reading and loading the json files

In [ ]:
# create function that reads cleans and appends json datasets to the general dataframe
def json_read_clean_append(json_file, duplicate_check, group_on, new_column, append_target=None):
    
    # first read json using geopandas
    temp_df = gpd.read_file(CSV_JSON_FOLDER + json_file)
    
    
    # load json into dataframe, drop any duplicate rows, count values for group_on series 
    temp_df = pd.DataFrame(temp_df).drop_duplicates(duplicate_check)[group_on].value_counts().to_frame(new_column)
    temp_df.index = pd.to_numeric(temp_df.index)
    
    if append_target is None:
        return temp_df
    else:
        return pd.merge(append_target, temp_df, 'left', left_index=True,right_index=True)

In [ ]:
# loop through json files in data folder and add to general df with function above
for file in listdir(CSV_JSON_FOLDER):
    if file.endswith('.json'):
        combined_df = json_read_clean_append(file,'adresse', 'plz', file[:-5], combined_df)

## Reading and loading the shape files

Shape files contain spatial information in form of point coordinates (longitude and latitude). In order to identify to which postal code they belong, we will be using zips.shp file that contains polygons describing area of zip codes in Switzerland. Furthermore, we will utilize geopandas functionality to join points with zip codes. One should also pay special attention to coordinate sysem. In order to join correctly all shape files should agree on coordinate system. Therefore, we will take coordinate systems used in zips.shp file as default and convert any other one into it for consistency.

In [ ]:
#read zips data
zips = gpd.read_file(SHAPE_FOLDER + "zips.shp")
zips.columns = ['UUID', 'OS_UUID', 'STATUS', 'INAEND', 'ZIP', 'ZUSZIFF','geometry']

#default coordinate system
COORDINATE_SYSTEM = zips.crs
print("used coordinate system: ", zips.crs)

Now we define method to read shape files into dataframe, to remove duplicates and convert coordinates to default coordinate system.

In [ ]:
#default coordinate system
COORDINATE_SYSTEM = {'init': 'epsg:4326'}

def read_shape_file(file_name):
    df = gpd.read_file(SHAPE_FOLDER + file_name)
    
    #drop duplicates
    G = df["geometry"].apply(lambda geom: geom.wkb)
    df = df.loc[G.drop_duplicates().index]
    
    #convert to default coordinate systen
    if (df.crs != COORDINATE_SYSTEM):
        df = df.to_crs(COORDINATE_SYSTEM)
    
    return df

Furthermore, we define method that joins points with corresponding polygons. Meaning we will much each coordinate (longitute and latitude) with corresponding zip code in Zurich and then groups counts by zip code.

In [ ]:
def join_points_with_zips(df):
    """join coordinates with zip codes"""
    pointInPoly = gpd.sjoin(df, zips, op='within') 
    return pointInPoly

In [ ]:
def grouping_by_zipcode(df, new_column):
    """group item counts per zip codes"""
    df = df.groupby('ZIP').size().reset_index(name=new_column)
    df.set_index('ZIP', inplace=True)
    return df

The method below will be used for appending information in the shape file to the existing central dataframe for aggregation of all information into one file.

In [ ]:
# create function that reads cleans and appends shape datasets to the general dataframe
def shape_read_clean_append(shape_file, new_column, append_target=None):
    
    # first read shape, drop duplicates
    temp_df = read_shape_file(shape_file)
    
    temp_df = join_points_with_zips(temp_df)
    
    # load shape into dataframe, drop any duplicate rows, count values for group_on series 
    temp_df = grouping_by_zipcode(temp_df, new_column)
    temp_df.index = pd.to_numeric(temp_df.index)
    #display(temp_df.head(2))
    
    if append_target is None:
        return temp_df
    else:
        return pd.merge(append_target, temp_df, 'left', left_index=True,right_index=True)

Now we will read all shape files from corresponding folder and load them into the central dataframe.

In [ ]:
# loop through shp files in data folder and add to general df with function above
for file in listdir(SHAPE_FOLDER):
    if file.endswith('.shp'):
        combined_df = shape_read_clean_append(file, file[:-4], combined_df)

("External Dataset Preprocessing.ipynb" contains some additional information about shape files)

## Final Merging

At the end we save combined information into .csv file for further usage.

In [ ]:
# drop zip codes that are not used (last 5 rows), and fill other nans with zeros
combined_df = combined_df.dropna(subset=['hospitality_companies']).fillna(0)
combined_df.to_csv(DATA_FOLDER + "open_data_aggregated.csv")
combined_df.head()

As we see datasets we will be using consists of:

- Addresses - 57923 in total
- Hospitality companies - 2097 in total
- Population - 509285 in total
- Beach volleyball - 12 in total
- Bike Parks - 1 in total
- Care centers - 20 in total
- Community centers - 18 in total
- Elementary schools - 115 in total
- Football fields - 15 in total
- Ice rinks - 3 in total
- Indoor pools - 7 in total
- Kindergartens - 241 in total
- Mobility rental - 227 in total
- Nurseries - 316 in total
- Outdoor pools - 6 in total
- Police locations - 28 in total
- Skate parks - 8 in total
- Tennis courts - 8 in total
- Parks - 118 in total
- Street Lights (Beleuchtung) - 40010 in total
- Old people center (Alterszentrum) - 28 in total
- Retirment houses (Alterswohnung) - 36 in total
- Handicapped parking (Behindertenparkplatz) - 410 in total
- Fontaines (brunnen) - 1281 in total
- Youth clubs/centers/ meeting center (Jugendtreff) - 13 in total
- Kindergarten - 354 in total
- Kinderhaus / Eltern-Kind-Zentrum - 16 in total
- Churches (Kirche) - 82 in total
- Temporary art in the urban space (Kunst im Stadtraum) - 388 in total
- picknickplatz - 110 in total
- social center (sozial zentrum) - 5 in total
- Gym (Sporthalle) - 16 in total
- Stadium - 1 in total (therefore won't be using, unless for combined sports metric)
- WC handicapped - 28 in total
- WC not handicapped - 77 in total

## Creating aggregated and normalized features

We combine features to create the following columns: sports_facilities, child_facilities, elderly_facilities, parks, toilets, handicapped_facilities (handicapp), transport_rental, community_facilities

In [ ]:
combined_df.columns

In [ ]:
# aggregate features and rename others to english
aggregated_df = combined_df[['addresses', 'hospitality_companies', 'population','police_locations']]
aggregated_df = aggregated_df.assign(parks=combined_df[['Park', 'Picknickplatz']].sum(axis=1))

aggregated_df = aggregated_df.assign(sport_facilities=combined_df\
                                    [['indoor_pools', 'outdoor_pools','ice_rinks', 'skate_parks','football_fields',\
                                    'tennis_courts','bikeparks','beachvolleyball','Stadion', 'Sporthalle']].sum(axis=1))

aggregated_df = aggregated_df.assign(child_facilities=combined_df[['elementary_schools', 'nurseries','kindergartens',\
                                    'Kinderhaus']].sum(axis=1))

aggregated_df = aggregated_df.assign(elderly_facilities=combined_df[['care_centers', 'Alterswohnung','Alterszentrum']]\
                                     .sum(axis=1))

aggregated_df = aggregated_df.assign(toilets=combined_df[['ZueriWC_nichtrollstuhlgaengig', 'ZueriWC_rollstuhlgaengig']]\
                                     .sum(axis=1))

aggregated_df = aggregated_df.assign(handicapp=combined_df[['Behindertenparkplatz', 'ZueriWC_rollstuhlgaengig']]\
                                     .sum(axis=1))

aggregated_df = aggregated_df.assign(transport_rental=combined_df[['Mobility_rental', 'publibike']]\
                                     .sum(axis=1))

aggregated_df = aggregated_df.assign(community_facilities=combined_df[['community_centers', 'Jugendtreff']]\
                                     .sum(axis=1))


aggregated_df = aggregated_df.assign(fountains=combined_df['Brunnen'])
aggregated_df = aggregated_df.assign(street_art=combined_df['KunstImStadtraum'])
aggregated_df = aggregated_df.assign(social_centers=combined_df['Sozialzentrum'])
aggregated_df = aggregated_df.assign(street_lights=combined_df['Beleuchtung'])
aggregated_df = aggregated_df.assign(churches=combined_df['Kirche'])
aggregated_df.head()

In [ ]:
# normalizing by population
pop_normed_df = aggregated_df.div(aggregated_df.population, axis=0).drop(columns='population')
pop_normed_df.head()

In [ ]:
# extract geometries for each zip and join on zip to aggregated_df. Create new dataframe with aggregated data and geom
aggregated_df2 = aggregated_df
aggregated_df2 = pd.merge(aggregated_df2, zips[['ZIP', 'geometry']].set_index('ZIP'), 'inner', left_index=True,right_index=True)
aggregated_df2 = aggregated_df2.drop_duplicates('addresses')

# get geopandas df
zurich_zips_geoms = gpd.GeoDataFrame(aggregated_df2.geometry)

# use coordinate system of shape file
zurich_zips_geoms.crs = {'init': 'epsg:4326'}

# project to equal area cylindrical and calculate area in square kilometers
zips_areas = zurich_zips_geoms.geometry.to_crs({'proj':'cea'}).map(lambda p: p.area / 10**6)

# create new dataframe with aggregated data normalized by area 
area_normed_df = aggregated_df.div(zips_areas, axis=0)
area_normed_df.head()

In [ ]:
point1 = Point(50.67, 4.62)
point2 = Point(51.67, 4.64)

geod = pyproj.Geod(ellps='WGS84')
angle1,angle2,distance = geod.inv(point1.x, point1.y, point2.x, point2.y)
print(distance/1000)

In [93]:
distance_dictionary = { "parks": ['Park.shp', 
                                  'Picknickplatz.shp'],
                       
                        "police_locations": ['police_locations.json'],
                       
                        "toilets": ['ZueriWC_nichtrollstuhlgaengig.shp', 
                                    'ZueriWC_rollstuhlgaengig.shp'],
                                    
                        "handicapp": ['ZueriWC_rollstuhlgaengig.shp', 
                                      'Behindertenparkplatz.shp'],
                                    
                        "hospitality_companies": ['gastwirtschaftsbetriebe_per_20171231.csv']}


def find_distance_to_closest_point(origin, points):
    """Returns distance from origin to closest point in points
    """
    
    geod = pyproj.Geod(ellps='WGS84')
    angle1, angle2, distance = geod.inv(origin.x, origin.y, points.iloc[0].x, points.iloc[0].y)
    min_distance = distance
    #min_distance = origin.distance(points.iloc[0])
    
    for point in points:
        angle1, angle2, distance = geod.inv(origin.x, origin.y, point.x, point.y)
        if (distance < min_distance):
            min_distance = distance
            
    return min_distance


def create_closest_distance_df(buildings, destinations, column_name):
    """Returns buildings dataframe containing info about distance to closest point in destinations for each row (Building)
    """
    #print(buildings)
    #display(destinations)
    
    distances = []
    for index, building in buildings.iterrows():
        
        distance = find_distance_to_closest_point(building['geometry'], destinations)
        #print(index, distance)
        distances.append(distance)
        #distances.append(5)
     
    print("distance array size", len(distances))
    #print(distances)
    
    buildings[column_name + "_distance"] = distances
    return buildings

def closest_distance(distance_dictionary):
    
    #read building file and create coordinates for each building for finding distances
    buildings = pd.read_csv(CSV_JSON_FOLDER + "adressen.csv")
    buildings['geometry'] = [Point(buildings.loc[i, 'easting_wgs'], 
                                   buildings.loc[i, 'northing_wgs']) for i in range(buildings.shape[0])]
    buildings = buildings[['plz', 'geometry']]
    #buildings.index = buildings.plz
    #buildings.drop(['plz'], axis=1, inplace=True)
    display(buildings.head())
    print("Shape of buildings: ", buildings.shape)
    
    for metric, files in distance_dictionary.items():
        merged_df = None

        for file in files:
            if file.endswith('.json'):
                temp_df = gpd.read_file(CSV_JSON_FOLDER + file)['geometry']
            if file.endswith('.csv'):
                temp_df = pd.read_csv(CSV_JSON_FOLDER + file)
                temp_df['geometry'] = [Point(temp_df.loc[i, 'X_WGS84'], 
                                             temp_df.loc[i, 'Y_WGS84']) for i in range(temp_df.shape[0])]
                temp_df = temp_df['geometry'] 
            if file.endswith('.shp'):
                temp_df = read_shape_file(file)['geometry']
                
            if (merged_df is None):
                merged_df = temp_df
            else:
                merged_df = pd.concat([merged_df, temp_df], axis=0).reset_index(drop=True)
           
        merged_df.name = metric
        display(merged_df.head())
        print("shape of metric",metric, merged_df.shape)
        
        buildings = create_closest_distance_df(buildings, merged_df, metric) 
        display(buildings.head())
        
    return buildings
        
    
buildings = closest_distance(distance_dictionary)

,plz,geometry
0,8048,POINT (8.505701286647051 47.38321309177221)
1,8048,POINT (8.478133315673031 47.3907974916887)
2,8006,POINT (8.544878299474929 47.3928419137778)
3,8064,POINT (8.477342133766768 47.4001601473413)
4,8064,POINT (8.47750095208948 47.40040407573461)


Shape of buildings:  (58260, 2)


0      POINT (8.50940278693184 47.3795806101971)
1    POINT (8.534235628571883 47.41123530013046)
2    POINT (8.513664780449393 47.33483506723053)
3    POINT (8.532626779342227 47.35615539797409)
4    POINT (8.498499038481786 47.42393678597789)
Name: parks, dtype: object

shape of metric parks (228,)
distance array size 58260


,plz,geometry,parks_distance
0,8048,POINT (8.505701286647051 47.38321309177221),409.433469
1,8048,POINT (8.478133315673031 47.3907974916887),1057.558509
2,8006,POINT (8.544878299474929 47.3928419137778),588.519043
3,8064,POINT (8.477342133766768 47.4001601473413),780.338745
4,8064,POINT (8.47750095208948 47.40040407573461),768.746264


...

shape of metric handicapp (487,)
distance array size 58260


,plz,geometry,parks_distance,police_locations_distance,toilets_distance,handicapp_distance
0,8048,POINT (8.505701286647051 47.38321309177221),409.433469,1121.332717,506.483311,73.561838
1,8048,POINT (8.478133315673031 47.3907974916887),1057.558509,740.506532,71.635238,71.635238
2,8006,POINT (8.544878299474929 47.3928419137778),588.519043,536.870557,499.852502,221.838226
3,8064,POINT (8.477342133766768 47.4001601473413),780.338745,1618.240347,436.093134,662.425931
4,8064,POINT (8.47750095208948 47.40040407573461),768.746264,1638.307364,442.689598,632.798164


0      POINT (8.5004599209453 47.3918199480363)
1      POINT (8.5004599209453 47.3918199480363)
2    POINT (8.498509549776781 47.3923896087339)
3     POINT (8.49889506040714 47.4016426347132)
4      POINT (8.5329884834744 47.3830914337617)
Name: hospitality_companies, dtype: object

shape of metric hospitality_companies (2224,)
distance array size 58260


,plz,geometry,parks_distance,police_locations_distance,toilets_distance,handicapp_distance,hospitality_companies_distance
0,8048,POINT (8.505701286647051 47.38321309177221),409.433469,1121.332717,506.483311,73.561838,38.745401
1,8048,POINT (8.478133315673031 47.3907974916887),1057.558509,740.506532,71.635238,71.635238,30.831961
2,8006,POINT (8.544878299474929 47.3928419137778),588.519043,536.870557,499.852502,221.838226,189.351574
3,8064,POINT (8.477342133766768 47.4001601473413),780.338745,1618.240347,436.093134,662.425931,29.678371
4,8064,POINT (8.47750095208948 47.40040407573461),768.746264,1638.307364,442.689598,632.798164,4.065005


In [94]:
buildings.shape

(58260, 7)

In [97]:
buildings.to_csv(DATA_FOLDER + "buildings_distance.csv")

## Exploring some of the Zurich open data

In [ ]:
#read zips data
zips = gpd.read_file(SHAPE_FOLDER + "zips.shp")
zips.columns = ['UUID', 'OS_UUID', 'STATUS', 'INAEND', 'ZIP', 'ZUSZIFF','geometry']

In [ ]:
# extract only geometries and zips and join on zurich zips 
area_normed_df2 = area_normed_df
area_normed_df2 = pd.merge(area_normed_df2, zips[['ZIP', 'geometry']].set_index('ZIP'), 'inner', left_index=True,right_index=True)
area_normed_df2 = area_normed_df2.drop_duplicates('addresses')

In [ ]:
def plot_geopandas(gp, col_to_plot,title,colormap):
    '''
    This function creates a choropleth map using geopandas
    
    Parameters:
    gp = geodataframe data containing geometry column
    col_to_plot = column of data that must be plotted
    title = title of the map
    colormap = plt colormap to use, e.g. 'Blues', 'Accent', ...
    '''
    f, ax = plt.subplots(figsize=(15, 10))
    
    from mpl_toolkits.axes_grid1 import make_axes_locatable

    # Create colorbar
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.2)
    
    # plotting settings
    ax.set_title(title, fontdict={'fontsize': '25', 'fontweight' : '3'})
    ax.set_aspect('equal')
    cax.set_aspect('equal')
    cax.set_axis_off()
    ax.set_axis_off()

    # get preferred colormap
    cmap = plt.cm.get_cmap(colormap)
    
    # plot data
    gp.plot(column=col_to_plot, ax=cax, linewidth=0.1, edgecolor='black',cmap=cmap)
    
    # create colorbar
    norm=plt.Normalize(vmin=gp[col_to_plot].min(), vmax=gp[col_to_plot].max())
    sm = plt.cm.ScalarMappable(cmap=cmap,norm=norm)
    sm._A = []
    f.colorbar(sm)
    
# use function defined above to create choropleth map 
plot_geopandas(gpd.GeoDataFrame(area_normed_df2), 'population', 'Population density in Zurich postal areas','Blues')

As we can see, there is some disparity in population distribution around Zurich. The northern areas seem to be more residential areas, whereas the southern areas have relatively less residents. Let's now take a look at the areas with the most hospitalitys companies, i.e. bars, cafes, restaurants etc..

In [ ]:
# extract only geometries and zips and join on zurich zips 
pop_normed_df2 = pop_normed_df
pop_normed_df2 = pd.merge(pop_normed_df2, zips[['ZIP', 'geometry']].set_index('ZIP'), 'inner', left_index=True,right_index=True)
pop_normed_df2 = pop_normed_df2.drop_duplicates('addresses')

In [ ]:
# use function defined above to create choropleth map 
plot_geopandas(gpd.GeoDataFrame(pop_normed_df2), 'hospitality_companies',\
               'Number of hospitality companies per person in  per person Zurich postal areas','Purples')

We see here that the northern areas have more restaurants, bars etc.. This could be linked to the fact that more people live there, as seen previously. Let's also take a look at which areas have the most addresses, this could be an indication of building density.

In [ ]:
# use function defined above to create choropleth map 
plot_geopandas(gpd.GeoDataFrame(pop_normed_df2), 'addresses',\
               'Number of building addresses per person in Zurich postal areas','Oranges')

Once again the same areas are highlighted. This brief exploration allows us to get an overview of the layout of Zurich, and how people and buildings are dispersed. A more detailed analysis is expected for the next milestone of our project.